In [5]:
# CHALLENGE 1: CONVERT HEX TO B64
from base64 import b16decode, b64encode

def hex_to_b64(data_hex: bytes) -> bytes:
    return b64encode(b16decode(data_hex, casefold=True))

data_hex = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"
data_b64 = hex_to_b64(data_hex)

print(f"{data_hex=}")
print(f"{data_b64=}")

if data_b64 == b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t':
    print("It worked!")
else:
    exit("Conversion failed")

data_hex='49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
data_b64=b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'
It worked!


In [24]:
# CHALLENGE 2: FIXED XOR
def xor(a: str, b:str) -> str:
    if len(a) != len(b):
        raise ValueError("Input strings are not of equal length!")

    return hex(int(a, 16) ^ int(b, 16))[2:]
    
hex_1 = '1c0111001f010100061a024b53535009181c'
hex_2 = '686974207468652062756c6c277320657965'
answer = '746865206b696420646f6e277420706c6179'

result = xor(hex_1, hex_2)

print(f"Correct? (True or False?): {answer == result}\nResult: {result}")

Correct? (True or False?): True
Result: 746865206b696420646f6e277420706c6179


In [3]:
# CHALLENGE 3 : SINGLE-BYTE XOR CIPHER
def single_byte_xor_cipher(hex_string):
    """ Converting hex string to bytes """
    bytes_data = bytes.fromhex(hex_string)
    results = []
    
    for key in range(256):
        decrypted = bytes([byte ^ key for byte in bytes_data])
        score = sum([chr(byte).lower() in 'etaoin shrldu' for byte in decrypted])
        results.append((score, key, decrypted.decode('utf-8', 'ignore')))
    return max(results, key=lambda x: x[0])

hex_string = "1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736"
result = single_byte_xor_cipher(hex_string)
print("Key:", result[1])
print("Decrypted message:", result[2])

Key: 88
Decrypted message: Cooking MC's like a pound of bacon


In [9]:
# CHALLENGE 4 : DETECT-SINGLE CHARACTER XOR
from utils import xor

def parse_input(filename):
    with open(filename) as f:
        cts = f.read().split('\n')

    return [bytes.fromhex(ct) for ct in cts]

def find_xored_ct(cts: list) -> bytes:
    best_ofs = []
    for ct in cts:
        # Use your existing single_byte_xor_cipher function
        result = single_byte_xor_cipher(ct.hex())
        best_ofs.append((result[0], result[1], ct, result[2]))

    best_ofs.sort(key=lambda x:x[0], reverse=True)

    return best_ofs[0]

def main():
    cts = parse_input('./challenge4data.txt')

    winner = find_xored_ct(cts)
    print(winner)
    key = bytes([winner[1]])
    pt = winner[2]

    print(f"pt: {xor(pt, key)}")

if __name__ == '__main__':
    main()

(22, 53, b'{ZB\x15A]TA\x15A]P\x15ETGAL\x15\\F\x15_@XE\\[R?', 'Now that the party is jumping\n')
pt: b'N'


In [19]:
# CHALLENGE 5 : IMPLEMENT REPEATING-KEY XOR
import binascii

def main():
    stanza = b"Burning 'em, if you ain't quick and nimble\nI go crazy when I hear a cymbal"
    key = b'ICE'
    answer = '0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f'

    result = xor(stanza, key)
    result = binascii.hexlify(result).decode()

    print(f"Correct? (True or False?): {answer == result}\nResult: {result}")

if __name__ == '__main__':
    main()

Correct? (True or False?): False
Result: 0b3637


In [59]:
# CHALLENGE 6 : BREAK REPEATING-KEY XOR
from base64 import b64decode
from utils1 import get_single_byte_xor

def parse_input(filename):
    with open(filename) as f:
        data = f.read().replace('\n', '')

    return b64decode(data.encode())

def hamming_distance(a: bytes, b: bytes) -> int:
    return sum([bin(b1 ^ b2).count('1') for b1, b2 in zip(a,b)])

def get_key(ct: bytes, size: int) -> bytes:
    key = b''
    for i in range(size):
        block = ct[i:-1:size]
        key += get_single_byte_xor(block)

    return key

def get_key_size(ct: bytes) -> int:
    hamming_distances = [hamming_distance(ct[-i:] + ct[:-i], ct) for i in range(41)]

    return hamming_distances.index(min(hamming_distances[2:]))

def main():
    ct = parse_input('./resources/challenge6data.txt')
    key_size = get_key_size(ct)
    key = get_key(ct, key_size)
    pt = xor(ct, key)

    print(f"key: {key.decode()}\npt: {pt.decode()}")

if __name__ == '__main__':
    main()

key: Terminator X: Bring the noise
pt: I'm back and I'm ringin' the 
